In [ ]:
%reload_ext autoreload
%autoreload 2

import gif
import pylab as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from PIL import ImageColor
from matplotlib.colors import LinearSegmentedColormap
from pythonperlin import perlin, extend2d
import time

def remove_margins():
    """ Removes figure margins, keeps only plot area """
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0,0)
    return


## Generate perlin noise

In [ ]:
%%time
nx, ny = 25, 35
shape = (8, nx, ny)
p = perlin(shape, dens=32, seed=0)


## Helper functions

In [ ]:
def load_image(fname):
    image = Image.open(fname)
    img = image.convert("RGB")
    img = np.array(img).astype(float)
    img = np.sum(img, axis=2).T[:,::-1] / (3 * 255)
    img = (img > 0.5).astype(float)
    #img.shape, img.max(), img.min()
    return img


def split_into_blocks(x, nx, ny):
    blocks = np.hsplit(x, ny)
    blocks = np.vstack(blocks)
    blocks = np.split(blocks, nx * ny)
    return blocks


def calc_com(x, axis=0):
    com = 0.0
    if np.std(x):
        s = np.sum(x)
        d = x.shape[axis]
        i = (np.arange(d) + .5) / d - .5
        if axis:
            com = np.sum(np.dot(x, i), axis=0) / s
        else:
            com = np.sum(np.dot(x.T, i), axis=0) / s
    return com


def calc_fill_and_displacements(x, nx, ny):
    blocks = split_into_blocks(x, nx, ny)
    dx = []
    dy = []
    dz = []
    for b in blocks:
        dx.append(calc_com(b - 1, 0) - calc_com(b, 0))
        dy.append(calc_com(b - 1, 1) - calc_com(b, 1))
        dz.append(1 - np.sum(b) / b.size)
    dx = np.array(dx).reshape(ny, nx).T
    dy = np.array(dy).reshape(ny, nx).T
    dz = np.array(dz).reshape(ny, nx).T
    return dx, dy, dz


def silhuette_to_coords(fname, nx, ny):
    img = load_image(fname)
    x, y = np.meshgrid(np.arange(nx), np.arange(ny), indexing="ij")
    dx, dy, dz = calc_fill_and_displacements(img, nx, ny)
    return x, y, dx, dy, dz
    
def calc_displacement_levels(dx, dy):
    levels = (np.abs(dx) > 0) | (np.abs(dy) > 0)
    levels = levels.astype(float)
    level = np.max(levels) + 1
    start = True
    while start:
        start = False
        l_next = np.copy(levels)
        for i in [-1, 1]:
            for j in [-1, 1]:
                l = np.pad(levels, (1,1))
                l = np.roll(l, i, 0)
                l = np.roll(l, j, 1)
                l = l[1:-1][:,1:-1]
                l_next[l > 0] = 1
        mask = (levels == 0) & (l_next > 0)
        if np.any(mask):
            start = True
            levels[mask] = level
            level += 1
    return levels


## Draw wireframe
<br>

- To generate a smoothly morphing wireframe we use Perlin noise (no octaves) to distort each grid node, then populate the grid with more lines along one direction (x).
<br>

- Displace grid nodes to increase density based on edge detection of a black-and-white image of size 375 x 525. Then apply periodic interchange of noise and displacement.

In [ ]:
def plot_wireframe(i, fname, nx, ny, p):
    w = 2 * np.pi / 256
    x, y, dx, dy, dz = silhuette_to_coords(fname, nx, ny)
    p = p[i][16:][:,16:]
    p = p[::32][:,::32]
    z = np.exp(2j * np.pi * p)
    px, py = z.real, z.imag
    
    # Calc noisy x and y
    xn = x + 1.5 * px
    yn = y + 1.5 * py
    xn = extend2d(xn[:,::2][::2])
    yn = extend2d(yn[:,::2][::2])
    # Calc displaced x and y
    xd = x + 1.5 * dx #+ .2 * pslowx
    yd = y + 1.5 * dy #+ .2 * pslowy
    # Mix noisy and displaced x and y    
    levels = calc_displacement_levels(dx, dy)
    scale = 1 / np.power(levels, 0.25)
    f = ((levels <= 3) | (dz > 0)).astype(float)
    periodic = 0.8 + 0.2 * np.cos(w * i + np.pi)
    f = f * periodic * scale
    x = f * xd + (1 - f) * xn
    y = f * yd + (1 - f) * yn

    # Populate wires along x axis
    x = extend2d(x, 4, 0)
    y = extend2d(y, 4, 0)
    
    fig = plt.figure(figsize=(6,7.5), facecolor="#000022")
    remove_margins()
    cmap = LinearSegmentedColormap.from_list("cmap", ["dodgerblue", "white"])
    for i in range(x.shape[0]):
        color = cmap(i/x.shape[0])
        plt.plot(x[i], y[i], color=color)
    plt.xlim(-3,nx+3)
    plt.ylim(-1,ny+1)
    plt.tight_layout()
    return fig


nx, ny = 25, 35
fname = "image.png"
fig = plot_wireframe(128, fname, nx, ny, p)

## Animate wireframe using 1st dimension as a time axis

In [ ]:
# Set the dots per inch resolution
gif.options.matplotlib["dpi"] = 180

# Decorate a plot function with @gif.frame
@gif.frame
def plot(i, fname, nx, ny, p):
    plot_wireframe(i, fname, nx, ny, p)

# Construct "frames"
frames = [plot(i, fname, nx, ny, p) for i in range(0, 256, 2)]

# Save "frames" to gif with a specified duration (milliseconds) between each frame
gif.save(frames, 'wireframe.gif', duration=120)